In [93]:
import ijson
import json
import numpy as np
import pandas as pd
import os
from pandas.io.json import json_normalize

In [94]:
aws_json_path = './inputs/aws_FRIENDS.json'

## Celebrities:

In [95]:
with open(aws_json_path, 'r') as f:
    celebrities_list = list(ijson.items(f, 'Celebrities.item'))
df = json_normalize(celebrities_list)
df

,Timestamp,Celebrity.Urls,Celebrity.Name,Celebrity.Id,Celebrity.Confidence,Celebrity.Face.BoundingBox.Width,Celebrity.Face.BoundingBox.Height,Celebrity.Face.BoundingBox.Left,Celebrity.Face.BoundingBox.Top,Celebrity.Face.Landmarks,Celebrity.Face.Pose.Roll,Celebrity.Face.Pose.Yaw,Celebrity.Face.Pose.Pitch,Celebrity.Face.Quality.Brightness,Celebrity.Face.Quality.Sharpness,Celebrity.Face.Confidence
0,0,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,100,0.08958332985639572,0.15925925970077515,0.4437499940395355,0.10370370000600815,"[{'Type': 'eyeLeft', 'X': 0.4787929952144623, ...",8.76693058013916,19.623544692993164,13.070178031921387,61.156646728515625,78.74752044677734,99.9988784790039
1,0,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,50,0.08958332985639572,0.15925925970077515,0.5604166388511658,0.06111111119389534,"[{'Type': 'eyeLeft', 'X': 0.5902573466300964, ...",-4.817878246307373,4.505734920501709,15.243426322937012,37.00822067260742,78.74752044677734,99.99998474121094
2,458,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,98,0.08541666716337204,0.1518518477678299,0.42500001192092896,0.1388888955116272,"[{'Type': 'eyeLeft', 'X': 0.45719096064567566,...",7.912779808044434,18.563512802124023,13.607428550720215,62.63996887207031,53.47911071777344,100
3,458,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,100,0.09270833432674408,0.1648148149251938,0.5572916865348816,0.06851851940155029,"[{'Type': 'eyeLeft', 'X': 0.5872163772583008, ...",-5.437594413757324,6.162485122680664,12.458541870117188,35.924007415771484,67.36150360107422,99.9997787475586
4,959,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,89,0.09583333134651184,0.17037037014961243,0.4333333373069763,0.10925926268100739,"[{'Type': 'eyeLeft', 'X': 0.4698166847229004, ...",4.5922770500183105,18.148014068603516,9.707369804382324,57.66347122192383,73.4422378540039,99.99252319335938
5,959,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,98,0.09479166567325592,0.16851851344108582,0.5541666746139526,0.05740740895271301,"[{'Type': 'eyeLeft', 'X': 0.5855147838592529, ...",-4.781125545501709,5.3465189933776855,16.398733139038086,40.4273796081543,73.4422378540039,99.99996185302734
6,3962,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,76,0.14374999701976776,0.25555557012557983,0.684374988079071,0.10925926268100739,"[{'Type': 'eyeLeft', 'X': 0.7307764291763306, ...",-1.5205985307693481,-36.01525115966797,-3.724030017852783,42.46391296386719,60.634769439697266,99.99993896484375
7,4462,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,50,0.13437500596046448,0.23888888955116272,0.7552083134651184,0.07777778059244156,"[{'Type': 'eyeLeft', 'X': 0.7998675107955933, ...",0.32068321108818054,-44.05034637451172,-9.304539680480957,42.65411376953125,60.634769439697266,99.99922943115234
8,4963,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,95,0.13645833730697632,0.24259258806705475,0.7916666865348816,0.06296296417713165,"[{'Type': 'eyeLeft', 'X': 0.8360732793807983, ...",4.409728050231934,-46.21820831298828,-7.6900315284729,44.28529357910156,60.634769439697266,99.99725341796875
9,10468,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,97,0.16354165971279144,0.2907407283782959,0.41458332538604736,0.14629629254341125,"[{'Type': 'eyeLeft', 'X': 0.47582969069480896,...",3.535399913787842,26.62160873413086,23.563152313232422,47.364261627197266,78.74752044677734,99.99894714355469


In [96]:
def add_bounding_box(row):
    temp_dict = { 'left': row['Celebrity.Face.BoundingBox.Left'], \
                  'right': row['Celebrity.Face.BoundingBox.Left'] + row['Celebrity.Face.BoundingBox.Width'], \
                  'top' : row['Celebrity.Face.BoundingBox.Top'], \
                  'bottom' : row['Celebrity.Face.BoundingBox.Top'] + row['Celebrity.Face.BoundingBox.Height'] }
    return temp_dict

In [97]:
df.drop(['Celebrity.Face.Landmarks', 'Celebrity.Face.Pose.Roll', 'Celebrity.Face.Pose.Yaw', \
         'Celebrity.Face.Pose.Pitch', 'Celebrity.Face.Quality.Brightness', 'Celebrity.Face.Quality.Sharpness', \
         'Celebrity.Face.Confidence'], axis=1, inplace=True)
df['bounding_box'] = df.apply(add_bounding_box, axis=1)
#df['bounding_box.right'] = df['Celebrity.Face.BoundingBox.Left'] + df['Celebrity.Face.BoundingBox.Width']
#df['bounding_box.bottom'] = df['Celebrity.Face.BoundingBox.Top'] + df['Celebrity.Face.BoundingBox.Height']
#df = df.rename(columns = { "Celebrity.Face.BoundingBox.Top": "bounding_box.top",
#                           "Celebrity.Face.BoundingBox.Left": "bounding_box.left" })
df['timestamp'] = df['Timestamp']/1000
df['source'] = 'amazon'

df.drop(['Celebrity.Face.BoundingBox.Left', 'Celebrity.Face.BoundingBox.Width', 'Celebrity.Face.BoundingBox.Top',
         'Celebrity.Face.BoundingBox.Height', 'Timestamp'], axis=1, inplace=True)
df = df.rename(columns = { "Celebrity.Urls":"url", "Celebrity.Name": "name",
                           "Celebrity.Id": "id", "Celebrity.Confidence" : "confidence" })
df

,url,name,id,confidence,bounding_box,timestamp,source
0,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,100,"{'left': 0.4437499940395355, 'right': 0.533333...",0.000,amazon
1,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,50,"{'left': 0.5604166388511658, 'right': 0.649999...",0.000,amazon
2,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,98,"{'left': 0.42500001192092896, 'right': 0.51041...",0.458,amazon
3,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,100,"{'left': 0.5572916865348816, 'right': 0.650000...",0.458,amazon
4,[www.imdb.com/name/nm0001612],Matthew Perry,2i3Ga9e,89,"{'left': 0.4333333373069763, 'right': 0.529166...",0.959,amazon
5,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,98,"{'left': 0.5541666746139526, 'right': 0.648958...",0.959,amazon
6,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,76,"{'left': 0.684374988079071, 'right': 0.8281249...",3.962,amazon
7,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,50,"{'left': 0.7552083134651184, 'right': 0.889583...",4.462,amazon
8,[www.imdb.com/name/nm0001435],Lisa Kudrow,2aM9z3i,95,"{'left': 0.7916666865348816, 'right': 0.928125...",4.963,amazon
9,[www.imdb.com/name/nm0001455],Matt LeBlanc,43be2M,97,"{'left': 0.41458332538604736, 'right': 0.57812...",10.468,amazon


## Labels:

In [98]:
with open(aws_json_path, 'r') as f:
    labels_list = list(ijson.items(f, 'Labels.item'))
df = json_normalize(labels_list)
df

,Timestamp,Label.Name,Label.Confidence,Label.Instances,Label.Parents
0,0,Dungeon,53.042518615722656,[],[]
1,0,Human,98.46746063232422,[],[]
2,0,Person,98.46746063232422,[{'BoundingBox': {'Width': 0.19230295717716217...,[]
3,0,Prison,99.5436782836914,[],[]
4,0,Window,62.562015533447266,[],[]
...,...,...,...,...,...
847,29863,Logo,64.84605407714844,[],[{'Name': 'Symbol'}]
848,29863,Symbol,71.2292709350586,[],[]
849,29863,Text,87.5086669921875,[],[]
850,29863,Trademark,64.84605407714844,[],[{'Name': 'Symbol'}]


In [99]:
df['source'] = 'amazon'
df['timestamp'] = df['Timestamp']/1000
df = df.explode('Label.Instances').reset_index(drop=True)
df['Label.Instances'] = df['Label.Instances'].apply(lambda x: {} if pd.isna(x) else x)
df = df.join(json_normalize(df['Label.Instances']))
df['confidence'] = df.apply(lambda x: x['Confidence'] if pd.notna(x['Confidence']) else x['Label.Confidence'], axis = 1)

In [100]:
def handle_bounding_box_for_labels(row):
    if pd.isna(row['BoundingBox.Left']):
        return None
    temp_dict = { 'left': row['BoundingBox.Left'], \
                  'right': row['BoundingBox.Left'] + row['BoundingBox.Width'], \
                  'top' : row['BoundingBox.Top'], \
                  'bottom' : row['BoundingBox.Top'] + row['BoundingBox.Height'] }
    return temp_dict

In [101]:
df['bounding_box'] = df.apply(handle_bounding_box_for_labels, axis=1)
df.drop(['Timestamp', 'Label.Confidence', 'Confidence', 'Label.Instances', 'BoundingBox.Left', 'BoundingBox.Width', 
         'BoundingBox.Top', 'BoundingBox.Height'], axis=1, inplace=True)
df = df.rename(columns = { "Label.Name": "name", "Label.Parents": "parents"})
df

,name,parents,source,timestamp,confidence,bounding_box
0,Dungeon,[],amazon,0.000,53.042518615722656,None
1,Human,[],amazon,0.000,98.46746063232422,None
2,Person,[],amazon,0.000,98.38488006591797,"{'left': 0.3312615156173706, 'right': 0.523564..."
3,Person,[],amazon,0.000,98.06590270996094,"{'left': 0.6007581353187561, 'right': 0.791674..."
4,Person,[],amazon,0.000,96.37962341308594,"{'left': 0.38436955213546753, 'right': 0.55190..."
...,...,...,...,...,...,...
985,Logo,[{'Name': 'Symbol'}],amazon,29.863,64.84605407714844,None
986,Symbol,[],amazon,29.863,71.2292709350586,None
987,Text,[],amazon,29.863,87.5086669921875,None
988,Trademark,[{'Name': 'Symbol'}],amazon,29.863,64.84605407714844,None


## Logos:
Not supported


## Text/OCR:
Not supported.


## Speech:
Not Supported


## Shots:
Not supported.


## Content Moderation:

In [102]:
with open(aws_json_path, 'r') as f:
    moderation_list = list(ijson.items(f, 'ModerationLabels.item'))
df = json_normalize(moderation_list)
df

,Timestamp,ModerationLabel.Confidence,ModerationLabel.Name,ModerationLabel.ParentName
0,6965,67.12017059326172,Violence,
1,6965,67.12017059326172,Weapon Violence,Violence
2,7465,74.79823303222656,Violence,
3,7465,71.15692901611328,Weapon Violence,Violence
4,22981,66.28107452392578,Violence,


In [103]:
df['source'] = 'amazon'
df['Timestamp'] = df['Timestamp']/1000
df = df.rename(columns = { "Timestamp":"timestamp", "ModerationLabel.Name": "name", \
                           "ModerationLabel.Confidence" : "confidence", \
                           "ModerationLabel.ParentName" : "parent_category" })
df

,timestamp,confidence,name,parent_category,source
0,6.965,67.12017059326172,Violence,,amazon
1,6.965,67.12017059326172,Weapon Violence,Violence,amazon
2,7.465,74.79823303222656,Violence,,amazon
3,7.465,71.15692901611328,Weapon Violence,Violence,amazon
4,22.981,66.28107452392578,Violence,,amazon
